# Fire Service Efficiency Metric

Author: Yuxin Zhao

Last update: 27/05/2025

Dataset resources: West Midlands Fire Service (WMFS)

This notebook mainly focuses on:
 
1.	Data Preprocessing: Filter the West Midlands Fire Service (WMFS) incident dataset to retain only incidents classified as FIRE and FALSE_ALARM.

2.	Spatial Aggregation: Generate a 500-meter grid over the study area and spatially aggregate fire incidents within each grid cell.

3.	Timely Response Threshold: Define a timely fire response as one occurring within 300 seconds (5 minutes), in line with WMFS operational standards.

4.	Efficiency Metric: Develop a fire service efficiency metric based on the proportion of incidents responded to within the 300-second threshold.
	
5.	Spatial Analysis: Perform spatial analysis to examine the geographic distribution of fire incidents and assess the spatial variation in fire service efficiency across the region.
 

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Load the data

In [3]:
incidents = pd.read_excel("../Data/WMFS_datasets/wmfs_incidents.xlsx")

In [6]:
incidents = incidents[incidents['incident_classification_level1'].isin(['FIRE', 'FALSE_ALARM'])]
incidents

,call_time,incident_classification_label,incident_profile_label,incident_classification_level1,prl_count,brv_count,EASTINGS,NORTHINGS,call_seconds,reaction_seconds,driving_seconds
0,2009-01-01 00:00:39,False alarm raised with good intent,False Alarms,FALSE_ALARM,2,0,392062.102010,286844.969007,86,160,250
1,2009-01-01 00:10:45,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,405643.149442,277939.980158,85,65,239
2,2009-01-01 00:11:59,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410260.244387,288819.189377,62,106,115
3,2009-01-01 00:53:02,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,396779.250331,299030.106069,72,109,96
4,2009-01-01 00:52:13,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410667.961350,290492.478579,68,109,204
...,...,...,...,...,...,...,...,...,...,...,...
383662,2023-12-31 22:59:53,False Alarms (Equipment),Low Risk,FALSE_ALARM,2,0,398666.686740,283225.457018,88,82,281
383663,2023-12-31 23:11:53,False alarm raised with good intent,False Alarms,FALSE_ALARM,1,1,433136.867808,277909.030728,134,94,98
383664,2023-12-31 23:32:10,Accidental secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,0,1,401134.330416,277356.681953,136,152,227
383666,2023-12-31 23:49:50,False Alarms (Equipment),Low Risk,FALSE_ALARM,0,1,406221.058730,290654.181612,253,66,297


In [4]:
stations = pd.read_csv('../Data/WMFS_datasets/station_locations.csv')

### Aggregate the data into 500-m square grid

In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import numpy as np

# Read the data
incidents = gpd.GeoDataFrame(incidents, geometry=gpd.points_from_xy(incidents['EASTINGS'], incidents['NORTHINGS']))
stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations['Easting'], stations['Northing']))

# Set CRS for incidents and stations
incidents.set_crs(epsg=27700, inplace=True)
stations.set_crs(epsg=27700, inplace=True)

# Define the grid size
grid_size = 500


# Calculate the bounding box for the grid
xmin, ymin, xmax, ymax = incidents.total_bounds

# Calculate the bound
xmin = incidents['EASTINGS'].min() // grid_size * grid_size # // means quotient rounding
xmax = incidents['EASTINGS'].max() // grid_size * grid_size + grid_size # rounding down so another grid needs to be added
ymin = incidents['NORTHINGS'].min() // grid_size * grid_size
ymax = incidents['NORTHINGS'].max() // grid_size * grid_size + grid_size


In [6]:
# Create the grids for all of the cells 
x_coords = np.arange(xmin, xmax + grid_size, grid_size)
y_coords = np.arange(ymin, ymax + grid_size, grid_size)

grid_polygons = []
grid_ids = []

for i, x in enumerate(x_coords[:-1]):
    for j, y in enumerate(y_coords[:-1]):
        poly = Polygon([(x, y), (x + grid_size, y), (x + grid_size, y + grid_size), (x, y + grid_size)])
        grid_polygons.append(poly)
        grid_ids.append(f"{i}_{j}")

grid_gpd = gpd.GeoDataFrame({'grid_id': grid_ids, 'geometry': grid_polygons}, crs=incidents.crs)
# Construct the mapping table of grid_id -> grid_idx
grid_idx_map = pd.DataFrame({
    'grid_id': grid_gpd['grid_id'].values,
    'grid_idx': grid_gpd.index  
})

grid_gpd

,grid_id,geometry
0,0_0,"POLYGON ((386000 272000, 386500 272000, 386500..."
1,0_1,"POLYGON ((386000 272500, 386500 272500, 386500..."
2,0_2,"POLYGON ((386000 273000, 386500 273000, 386500..."
3,0_3,"POLYGON ((386000 273500, 386500 273500, 386500..."
4,0_4,"POLYGON ((386000 274000, 386500 274000, 386500..."
...,...,...
7592,106_66,"POLYGON ((439000 305000, 439500 305000, 439500..."
7593,106_67,"POLYGON ((439000 305500, 439500 305500, 439500..."
7594,106_68,"POLYGON ((439000 306000, 439500 306000, 439500..."
7595,106_69,"POLYGON ((439000 306500, 439500 306500, 439500..."


In [7]:
grid_idx_map

,grid_id,grid_idx
0,0_0,0
1,0_1,1
2,0_2,2
3,0_3,3
4,0_4,4
...,...,...
7592,106_66,7592
7593,106_67,7593
7594,106_68,7594
7595,106_69,7595


In [8]:
# centroid of all the grid cells 
from pathlib import Path

data_dir = Path("fire_station_optimisation_ga/data")
# Ensure the directory exists
data_dir.mkdir(parents=True, exist_ok=True)

# Calculate the centroid of each grid 
grid_gpd['centroid'] = grid_gpd.geometry.centroid

# Extract as NumPy array (cells, 2)
xy_all = np.vstack([
    grid_gpd['centroid'].x.values,
    grid_gpd['centroid'].y.values
]).T    # shape: (num_cells, 2)


# Save as xy_all.npy
np.save(data_dir / "xy_all.npy", xy_all)

print(" xy_all is complete，shape =", xy_all.shape)

xy_all

 xy_all is complete，shape = (7597, 2)


array([[386250., 272250.],
       [386250., 272750.],
       [386250., 273250.],
       ...,
       [439250., 306250.],
       [439250., 306750.],
       [439250., 307250.]])

In [9]:
# Ensure index_right does not exist before sjoin
incidents.drop(columns=['index_right'], errors='ignore', inplace=True)
stations.drop(columns=['index_right'], errors='ignore', inplace=True)

# Perform spatial join
incidents = gpd.sjoin(incidents, grid_gpd, how="left", predicate="intersects")
stations = gpd.sjoin(stations, grid_gpd, how="left", predicate="intersects")

#incidents.drop(columns=['grid_id_left'])

incidents

,call_time,incident_classification_label,incident_profile_label,incident_classification_level1,prl_count,brv_count,EASTINGS,NORTHINGS,call_seconds,reaction_seconds,driving_seconds,geometry,index_right,grid_id,centroid
0,2009-01-01 00:00:39,False alarm raised with good intent,False Alarms,FALSE_ALARM,2,0,392062.102010,286844.969007,86,160,250,POINT (392062.102 286844.969),881,12_29,POINT (392250 286750)
1,2009-01-01 00:10:45,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,405643.149442,277939.980158,85,65,239,POINT (405643.149 277939.98),2780,39_11,POINT (405750 277750)
2,2009-01-01 00:11:59,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410260.244387,288819.189377,62,106,115,POINT (410260.244 288819.189),3441,48_33,POINT (410250 288750)
3,2009-01-01 00:53:02,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,396779.250331,299030.106069,72,109,96,POINT (396779.25 299030.106),1545,21_54,POINT (396750 299250)
4,2009-01-01 00:52:13,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410667.961350,290492.478579,68,109,204,POINT (410667.961 290492.479),3515,49_36,POINT (410750 290250)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383663,2023-12-31 23:11:53,False alarm raised with good intent,False Alarms,FALSE_ALARM,1,1,433136.867808,277909.030728,134,94,98,POINT (433136.868 277909.031),6685,94_11,POINT (433250 277750)
383664,2023-12-31 23:32:10,Accidental secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,0,1,401134.330416,277356.681953,136,152,227,POINT (401134.33 277356.682),2140,30_10,POINT (401250 277250)
383665,2023-12-31 23:34:28,"Water incident (flooding, leaks, rescues etc.)",Low Risk,SSC,1,0,392245.881187,283026.990442,119,88,307,POINT (392245.881 283026.99),874,12_22,POINT (392250 283250)
383666,2023-12-31 23:49:50,False Alarms (Equipment),Low Risk,FALSE_ALARM,0,1,406221.058730,290654.181612,253,66,297,POINT (406221.059 290654.182),2877,40_37,POINT (406250 290750)


In [11]:
incident_xy = incidents[['EASTINGS', 'NORTHINGS']].to_numpy()
data_dir = Path("fire_station_optimisation_ga/data")
data_dir.mkdir(exist_ok=True)

np.save(data_dir / "incident_xy.npy", incident_xy)


In [12]:
incident_grid_idx = incidents['index_right'].to_numpy()
data_dir = Path("fire_station_optimisation_ga/data")
data_dir.mkdir(exist_ok=True)

np.save(data_dir / "incident_grid_idx", incident_grid_idx)


In [13]:
# Drop unnecessary columns after the join
incidents.drop(columns=['index_right'], errors='ignore', inplace=True)
# Handle NaN values in grid_id
incidents['grid_id'].fillna('-1_-1', inplace=True)  # Assign placeholder for NaN

# Split grid_id into grid_x and grid_y and convert to int
incidents[['grid_x', 'grid_y']] = incidents['grid_id'].str.split('_', expand=True)
incidents['grid_x'] = pd.to_numeric(incidents['grid_x'], errors='coerce').fillna(-1).astype('int64')
incidents['grid_y'] = pd.to_numeric(incidents['grid_y'], errors='coerce').fillna(-1).astype('int64')

incidents

/var/folders/_h/2n_x6vl97r9b0c9md5p0zcgm0000gn/T/ipykernel_39866/2654664705.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  incidents['grid_id'].fillna('-1_-1', inplace=True)  # Assign placeholder for NaN


,call_time,incident_classification_label,incident_profile_label,incident_classification_level1,prl_count,brv_count,EASTINGS,NORTHINGS,call_seconds,reaction_seconds,driving_seconds,geometry,grid_id,centroid,grid_x,grid_y
0,2009-01-01 00:00:39,False alarm raised with good intent,False Alarms,FALSE_ALARM,2,0,392062.102010,286844.969007,86,160,250,POINT (392062.102 286844.969),12_29,POINT (392250 286750),12,29
1,2009-01-01 00:10:45,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,405643.149442,277939.980158,85,65,239,POINT (405643.149 277939.98),39_11,POINT (405750 277750),39,11
2,2009-01-01 00:11:59,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410260.244387,288819.189377,62,106,115,POINT (410260.244 288819.189),48_33,POINT (410250 288750),48,33
3,2009-01-01 00:53:02,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,396779.250331,299030.106069,72,109,96,POINT (396779.25 299030.106),21_54,POINT (396750 299250),21,54
4,2009-01-01 00:52:13,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410667.961350,290492.478579,68,109,204,POINT (410667.961 290492.479),49_36,POINT (410750 290250),49,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383662,2023-12-31 22:59:53,False Alarms (Equipment),Low Risk,FALSE_ALARM,2,0,398666.686740,283225.457018,88,82,281,POINT (398666.687 283225.457),25_22,POINT (398750 283250),25,22
383663,2023-12-31 23:11:53,False alarm raised with good intent,False Alarms,FALSE_ALARM,1,1,433136.867808,277909.030728,134,94,98,POINT (433136.868 277909.031),94_11,POINT (433250 277750),94,11
383664,2023-12-31 23:32:10,Accidental secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,0,1,401134.330416,277356.681953,136,152,227,POINT (401134.33 277356.682),30_10,POINT (401250 277250),30,10
383666,2023-12-31 23:49:50,False Alarms (Equipment),Low Risk,FALSE_ALARM,0,1,406221.058730,290654.181612,253,66,297,POINT (406221.059 290654.182),40_37,POINT (406250 290750),40,37


In [14]:
incidents = incidents.merge(grid_idx_map, on='grid_id', how='left')
incidents['grid_idx'] = incidents['grid_idx'].fillna(0).astype(int) 

In [15]:
incidents['total_response_time'] = incidents['reaction_seconds'] + incidents['driving_seconds']

In [16]:
# The threshold for response time in seconds
response_threshold = 300
# Add a new column: Whether the response time exceeds 300 seconds (1: exceeds, 0: not exceeds)
incidents['over_response_threshold'] = (incidents['total_response_time'] > response_threshold).astype(int)
incidents

,call_time,incident_classification_label,incident_profile_label,incident_classification_level1,prl_count,brv_count,EASTINGS,NORTHINGS,call_seconds,reaction_seconds,driving_seconds,geometry,grid_id,centroid,grid_x,grid_y,grid_idx,total_response_time,over_response_threshold
0,2009-01-01 00:00:39,False alarm raised with good intent,False Alarms,FALSE_ALARM,2,0,392062.102010,286844.969007,86,160,250,POINT (392062.102 286844.969),12_29,POINT (392250 286750),12,29,881,410,1
1,2009-01-01 00:10:45,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,405643.149442,277939.980158,85,65,239,POINT (405643.149 277939.98),39_11,POINT (405750 277750),39,11,2780,304,1
2,2009-01-01 00:11:59,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410260.244387,288819.189377,62,106,115,POINT (410260.244 288819.189),48_33,POINT (410250 288750),48,33,3441,221,0
3,2009-01-01 00:53:02,Secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,1,0,396779.250331,299030.106069,72,109,96,POINT (396779.25 299030.106),21_54,POINT (396750 299250),21,54,1545,205,0
4,2009-01-01 00:52:13,False alarm raised with malicious intent,False Alarms,FALSE_ALARM,1,0,410667.961350,290492.478579,68,109,204,POINT (410667.961 290492.479),49_36,POINT (410750 290250),49,36,3515,313,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292692,2023-12-31 22:59:53,False Alarms (Equipment),Low Risk,FALSE_ALARM,2,0,398666.686740,283225.457018,88,82,281,POINT (398666.687 283225.457),25_22,POINT (398750 283250),25,22,1797,363,1
292693,2023-12-31 23:11:53,False alarm raised with good intent,False Alarms,FALSE_ALARM,1,1,433136.867808,277909.030728,134,94,98,POINT (433136.868 277909.031),94_11,POINT (433250 277750),94,11,6685,192,0
292694,2023-12-31 23:32:10,Accidental secondary fire,Secondary Fires that attract a 20 minute-respo...,FIRE,0,1,401134.330416,277356.681953,136,152,227,POINT (401134.33 277356.682),30_10,POINT (401250 277250),30,10,2140,379,1
292695,2023-12-31 23:49:50,False Alarms (Equipment),Low Risk,FALSE_ALARM,0,1,406221.058730,290654.181612,253,66,297,POINT (406221.059 290654.182),40_37,POINT (406250 290750),40,37,2877,363,1


In [17]:
from scipy.stats import shapiro
from scipy.stats import skew

grouped = incidents.groupby('grid_id')

# create a list for each unique grid id
# it includes grid id, the count of fire,
# the normality of the distribution of response time in each grid and the skewness of the distribution
distribution = []

for name, group in grouped:
    # count the fire incidents in each grid
    count = group.shape[0]
    
    # if the incidents are bigger than or equal to 3,
    # using Shapiro-Wilk for normality test
    if count >= 3:
        stat, p_value = shapiro(group['total_response_time'])
        
        normal = 'Yes' if p_value > 0.05 else 'No'

    else:
        
        normal = 'Insufficient data'
        
    grid_x = group['grid_x'].iloc[0]
    grid_y = group['grid_y'].iloc[0]
    
    distribution.append([name, grid_x, grid_y, count, normal])

grid = pd.DataFrame(distribution, columns=['grid_id', 'grid_x', 'grid_y', 'fire_count', 'is_normal'])
grid

,grid_id,grid_x,grid_y,fire_count,is_normal
0,0_51,0,51,1,Insufficient data
1,0_52,0,52,10,Yes
2,0_53,0,53,4,Yes
3,100_10,100,10,131,No
4,100_11,100,11,103,No
...,...,...,...,...,...
3729,9_59,9,59,27,No
3730,9_60,9,60,42,No
3731,9_61,9,61,63,No
3732,9_62,9,62,59,No


In [ ]:
# Calculate the number of events and total number of events within 300s in each grid_id
metrics = incidents.groupby('grid_id')['over_response_threshold'].agg(
    fires_under_300=lambda x: np.sum(x == 0),  # Calculate the number of events with response time ≤ 300s
    fires_over_300=lambda x: np.sum(x == 1),  # Calculate the number of events with response time > 300s
    total_fires=lambda x: len(x)  # Calculate the total number of fire events in this grid
)

# Calculate the proportion  (avoid the error of 0)
metrics['in_time_response_rate'] = metrics['fires_under_300'] / metrics['total_fires']

grid = grid.merge(metrics, on='grid_id', how='left')
grid = grid.merge(grid_idx_map, on='grid_id', how='left')

In [19]:
grid['fire_frequency'] = grid['fire_count'] / len(incidents)
grid

,grid_id,grid_x,grid_y,fire_count,is_normal,fires_under_300,fires_over_300,total_fires,in_time_response_rate,grid_idx,fire_frequency
0,0_51,0,51,1,Insufficient data,0,1,1,0.000000,51,0.000003
1,0_52,0,52,10,Yes,0,10,10,0.000000,52,0.000034
2,0_53,0,53,4,Yes,0,4,4,0.000000,53,0.000014
3,100_10,100,10,131,No,32,99,131,0.244275,7110,0.000448
4,100_11,100,11,103,No,55,48,103,0.533981,7111,0.000352
...,...,...,...,...,...,...,...,...,...,...,...
3729,9_59,9,59,27,No,0,27,27,0.000000,698,0.000092
3730,9_60,9,60,42,No,5,37,42,0.119048,699,0.000143
3731,9_61,9,61,63,No,3,60,63,0.047619,700,0.000215
3732,9_62,9,62,59,No,4,55,59,0.067797,701,0.000202


In [ ]:
partial_df = grid  

# total grids 
total_cells = 7597  # 你的 full grid size

# build a full DataFrame with all grid_idx from 0 to total_cells - 1
full_df = pd.DataFrame(index=np.arange(total_cells))

# Add grid_idx to the full DataFrame
full_df = full_df.merge(partial_df.set_index('grid_idx'), left_index=True, right_index=True, how='left')

# Fill NaN values with 0 for fire_count and in_time_response_rate, and 0.0 for fire_frequency
full_df['fire_count'] = full_df['fire_count'].fillna(0).astype(int)
full_df['in_time_response_rate'] = full_df['in_time_response_rate'].fillna(0.0)
full_df['fire_frequency'] = full_df['fire_frequency'].fillna(0.0)

# Reset the index and rename columns
full_df = full_df.reset_index().rename(columns={'index': 'grid_idx'})
full_df = full_df[['grid_idx', 'fire_count', 'in_time_response_rate', 'fire_frequency']]
full_df.to_csv("full_grid_metrics.csv", index=False)


In [21]:
# Save the incident frequency data
incident_freq = full_df['in_time_response_rate'].values
data_dir = Path("fire_station_optimisation_ga/data")
data_dir.mkdir(exist_ok=True)

np.save(data_dir / "incident_freq.npy", incident_freq)

### Current layout of the station ### 

In [22]:
stations

,Station name,Easting,Northing,PRL_Count,BRV_Count,Closed (Y/N),Opened,Closed,geometry,index_right,grid_id,centroid
0,Aldridge,405203,302818,1,0,N,Before study period,NaN,POINT (405203 302818),2759,38_61,POINT (405250 302750)
1,Aston,407316,289711,1,1,N,Before study period,NaN,POINT (407316 289711),3017,42_35,POINT (407250 289750)
2,Bickenhill,419627,284038,1,0,N,Before study period,NaN,POINT (419627 284038),4781,67_24,POINT (419750 284250)
3,Billesley,409029,281369,1,1,N,Before study period,NaN,POINT (409029 281369),3284,46_18,POINT (409250 281250)
4,Bilston,394543,296040,1,0,N,Before study period,NaN,POINT (394543 296040),1255,17_48,POINT (394750 296250)
5,Binley,436876,278691,1,1,N,Before study period,NaN,POINT (436876 278691),7184,101_13,POINT (436750 278750)
6,Bloxwich,399846,301797,1,0,N,Before study period,NaN,POINT (399846 301797),1976,27_59,POINT (399750 301750)
7,Bournbrook,405167,283287,1,0,N,Before study period,NaN,POINT (405167 283287),2720,38_22,POINT (405250 283250)
8,Brierley Hill,391862,287775,1,1,N,Before study period,NaN,POINT (391862 287775),812,11_31,POINT (391750 287750)
9,Canley,430542,277280,1,0,N,Before study period,NaN,POINT (430542 277280),6329,89_10,POINT (430750 277250)


In [23]:
from scipy.spatial import cKDTree

stations_xy = np.vstack([
    stations.geometry.x.values,
    stations.geometry.y.values
]).T  # shape: (num_stations, 2)

# Build KDTree index
tree = cKDTree(xy_all)

# Match stations to grid centroids
distances, indices = tree.query(stations_xy, k=1)
current_layout_idx = indices

# Save as xy_all.npy
np.save(data_dir / "current_layout_idx.npy", current_layout_idx)

current_layout_idx

array([2759, 3017, 4781, 3284, 1255, 7184, 1976, 2720,  812, 6329, 6688,
       1306,  962, 3590, 1051, 6907, 1376, 1299, 2591, 3575, 3079, 2642,
       2656, 2214, 1878, 3166, 4076, 2234, 3991,  592, 3741,  340, 1461,
       2112, 3583, 1749, 2027, 1616,  690, 2223])

### the features of the index without the distance ###

In [14]:
features = pd.read_csv("../Data/All_features.csv")
features.head()

,grid_id,in_time_response_rate,nearest_station_travel_time,"Accommodation, eating and drinking",Attractions,Commercial services,Education and health,Manufacturing and production,Public infrastructure,Retail,...,Low density residential with amenities (suburbs and small villages / hamlets),Medium density residential with high streets and amenities,Mining and spoil areas,Open or heath and moor land,Orchards,Principle Transport,Recreational land,Retail parks,Urban centres - mainly commercial/retail with residential pockets,Wetlands
0,0_51,0.000000,458.000000,0,0,0,0,0,0,0,...,1.56,0.000000e+00,0.0,0.0,0.0,1.340000e-09,0.00,0.0,0.0,0.0
1,0_52,0.000000,436.100000,0,0,0,1,0,1,0,...,29.52,3.960000e+00,0.0,0.0,0.0,5.560000e+00,0.00,0.0,0.0,0.0
2,0_53,0.000000,408.250000,0,1,0,1,1,1,0,...,40.48,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.00,0.0,0.0,0.0
3,100_10,0.244275,288.519084,0,2,1,2,0,4,0,...,5.20,1.790000e-09,0.0,0.0,0.0,9.840000e+00,31.96,0.0,0.0,0.0
4,100_11,0.533981,160.514563,0,1,3,0,0,1,0,...,17.16,3.192000e+01,0.0,0.0,0.0,6.920000e+00,8.32,0.0,0.0,0.0


In [15]:
# build a full DataFrame with all grid_idx from 0 to total_cells - 1
total_cells = 7597  # 或 grid_gpd.shape[0]
full_features = pd.DataFrame(index=np.arange(total_cells))

features = features.merge(grid_idx_map, on='grid_id', how='left')

full_features = full_features.merge(
    features.set_index('grid_idx'),  # 设置索引为 grid_idx
    left_index=True,
    right_index=True,
    how='left'  # 保留 full_features 的完整行
)

full_features = full_features.fillna(0)

full_features

,grid_id,in_time_response_rate,nearest_station_travel_time,"Accommodation, eating and drinking",Attractions,Commercial services,Education and health,Manufacturing and production,Public infrastructure,Retail,...,Low density residential with amenities (suburbs and small villages / hamlets),Medium density residential with high streets and amenities,Mining and spoil areas,Open or heath and moor land,Orchards,Principle Transport,Recreational land,Retail parks,Urban centres - mainly commercial/retail with residential pockets,Wetlands
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7592,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7593,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7594,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7595,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
Five_features = full_features[['neighbour_frequency_per_month', 
           'Agriculture - mainly crops', 
           'Deciduous woodland', 
           'station_count']]
Five_features 

,neighbour_frequency_per_month,Agriculture - mainly crops,Deciduous woodland,station_count
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
7592,0.0,0.0,0.0,0.0
7593,0.0,0.0,0.0,0.0
7594,0.0,0.0,0.0,0.0
7595,0.0,0.0,0.0,0.0


In [20]:
np.save(data_dir / "features_full.npy", full_features.values)
np.save(data_dir / "Five_features.npy", Five_features.values)